In [47]:
import os
import csv
import pandas as pd
import numpy as np
from census import Census

In [48]:
#todo when complete, delete this comment and uncomment the if, indent everything below
# if not os.path.exists('../input_data_clean/state_AZ-CA_clean.csv'):

#todo load the WHOLE thing when code is set
hmda = pd.read_csv('../input_data/state_AZ-CA_mini.csv')
hmda = hmda.replace(["Exempt","1111","8888","9999","NA"], np.nan)  

hmda['simplified_applicant_age_above_62'] = hmda['applicant_age_above_62'].map({'Yes': 1, 'No': 0})
hmda.drop(columns = ['applicant_age_above_62'],inplace=True)

#hmda['state_simplified'] = hmda['state_code'].map({'AZ': 1, 'CA': 0})

In [49]:
hmda["census_tract"] = hmda["census_tract"].astype(str)

In [50]:
hmda["census_tract"] = hmda["census_tract"].apply(lambda x: x[:-2])

In [51]:
# this csv: col1 is census var ID name, col2 is the name we are using
# we are importing this into a dictionary bc pandas rename() wants that structure
rename_dict ={}
with open('../input_data/census_vars.csv') as f:
    for row in csv.reader(f):
        rename_dict[row[0]] = row[1]
    

In [60]:
#todo when the below is done
# if not os.path.exists('../input_data_clean/census_clean.csv'):

#todo verifying the merge steps with juan...
census = (
        pd.read_csv('../input_data/census.csv')
        # are we sure? these are adding the #s... don't you mean to concatenate them as strings?
        #.assign(census_tract = lambda x: x["state"] + x["county"] + x["tract"])
        .rename(columns = rename_dict)
        # .drop(columns = ["state", "county", "tract"])
        #.to_csv('../input_data_clean/census_clean_sampel.csv',index=False)
)

#todo are these necessary?   depends on above
# census_clean["census_tract"] = census_clean["census_tract"].astype(str)
# census_clean["census_tract"] = census_clean["census_tract"].str.lstrip('0')
# census_clean["census_tract"] = census_clean["census_tract"].astype(int)

In [61]:
census["GEO_ID"] = census["GEO_ID"].astype(str)

In [62]:
census["GEO_ID"] = census["GEO_ID"].apply(lambda x: x.replace(x[:x.index("US0") + 3], ''))

In [64]:
hmda.rename(columns={"census_tract": "GEO_ID"}, inplace= True)

In [65]:
census["GEO_ID"]

0       6001400100
1       6001400200
2       6001400300
3       6001400400
4       6001400500
           ...    
9578    4027012100
9579    4027980003
9580    4027980004
9581    4027980005
9582    4027980006
Name: GEO_ID, Length: 9583, dtype: object

In [66]:
hmda["GEO_ID"]

0        6107003801
1        4019004328
2        6073019903
3        6109005100
4        6067002100
            ...    
49995    6065043240
49996    4013817600
49997    6037910202
49998    4019003200
49999    6001432100
Name: GEO_ID, Length: 50000, dtype: object

In [68]:
merged = hmda.merge(census, on = "GEO_ID", how = "left")

In [ ]:
def census_concat(row):
    state  = str(row['state']) .zfill(2)
    county = str(row['county']) .zfill(3)
    tract  = str(row['tract']) .zfill(6)
    return int(state+county+tract)

census['census_tract'] = census.apply(census_concat, axis=1)
census[['census_tract','state','county','tract']].sample(50)